In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio


In [2]:

load_dotenv(override=True)


True

In [3]:
attractions_planner_agent_instructions = """
You are a attractions planner. You are given a destination and need to plan all must-do things in that city.
You should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.
Attractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.
"""

activity_planner_agent_instructions = """
You are a activity planner. You are given a destination 
You need to find all local best activities in that city with their popularity.
Activities should be something to experience like skydiving, kayaking, sunset sails, snorkeling, etc.
"""

restaurant_planner_agent_instructions = """
You are a restaurant planner. You are given a destination and number of days to spend.
You need to find all local best restaurants in that city with their ratings and reviews. 
You need to plan for 3 meals a day for the number of days you are staying.
"""

attractions_planner_agent = Agent(
    name="Attractions Planner",
    instructions = attractions_planner_agent_instructions,
)

activity_planner_agent = Agent(
    name="Activity Planner",
    instructions = activity_planner_agent_instructions,
)

restaurant_planner_agent = Agent(
    name="Restaurant Planner",
    instructions = restaurant_planner_agent_instructions,
)



In [4]:
itenary_planner_agent_instructions = """
    You are given a list of attractions, activities and restaurants. 
    You need to pick best attractions, activities and restaurants out of the list to plan a trip to the destination.
    You give a day by day plan of the trip with timings for various attractions and activities.
    You'll also plan for appropriate schedules for 3 meals a day in nearby restaurants to the current schedule.
"""

itenary_planner_agent = Agent(
    name="Itenary Planner",
    instructions=itenary_planner_agent_instructions,
)

In [11]:

message = "SanDiego, 5 days"

with trace("Planning Itenary"):
    results = await asyncio.gather(
        Runner.run(attractions_planner_agent, message),
        Runner.run(activity_planner_agent, message),
        Runner.run(restaurant_planner_agent, message),
    )
    outputs = [result.final_output for result in results]

    todos = "Attractions, activities and restaurants:\n\n".join(outputs)

    plan = await Runner.run(itenary_planner_agent, todos)

    print(f"Best Plan:\n{plan.final_output}")



Best Plan:
Here's a 5-day itinerary for an exciting trip to San Diego, combining attractions, activities, and top-rated dining experiences. 

### Day 1: Beach and Ocean Adventures
- **Morning: La Jolla Cove Snorkeling**
  - Experience vibrant marine life and pristine waters.
- **Breakfast:** *The Cottage La Jolla*
  - Known for French toast and cozy ambiance.

- **Late Morning: Kayaking Tours in La Jolla Sea Caves**
  - Explore unique sea caves and cliffs.

- **Lunch:** *Hodad's Downtown*
  - Famous for enormous, delicious burgers.

- **Afternoon: Surfing at Pacific Beach**
  - Ride the consistent waves with local surf lessons.

- **Dinner:** *Born & Raised*
  - Enjoy a high-end steakhouse experience with elegant decor.

### Day 2: Outdoor Exploration
- **Morning: Hiking at Torrey Pines State Reserve**
  - Stunning ocean views and diverse wildlife.

- **Breakfast:** *Great Maple*
  - Innovative dishes with fantastic maple bacon doughnuts.

- **Late Morning: Biking around Mission Bay**


In [5]:
calender_generator_agent_instructions = """
You are a calender generator. You are given a ordered list of attractions, activities and restaurants with appropriate days and timings.
You need to generate a single ics file with all the events and their details.
You need to include all the details like name, description, start and end time, location, etc.
This ics file should be in a format that can be imported into any calendar app.
"""

calender_generator_agent = Agent(
    name="Calender Generator",
    instructions = calender_generator_agent_instructions,
)

In [6]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("harishjmhss@gmail.com")  # Change to your verified sender
    to_email = To("harishneyveli96@gmail.com")  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

# send_email("This is a test email")

In [7]:
activity_planner_tool = activity_planner_agent.as_tool(tool_name="activity_planner_tool", tool_description="Plan only activities for the given destination and number of days")
attractions_planner_tool = attractions_planner_agent.as_tool(tool_name="attractions_planner_tool", tool_description="Plan only attractions for the given destination and number of days")
restaurant_planner_tool = restaurant_planner_agent.as_tool(tool_name="restaurant_planner_tool", tool_description="Plan only restaurants for the given destination and number of days")
itenary_planner_tool = itenary_planner_agent.as_tool(tool_name="itenary_planner_tool", tool_description="Plan a trip to the given destination and number of days by calling activity planner, attractions planner and restaurant planner tools")
calender_generator_tool = calender_generator_agent.as_tool(tool_name="calender_generator_tool", tool_description="Generate a calender with all the events and their details")


In [8]:
tools = [
    activity_planner_tool,
    attractions_planner_tool,
    restaurant_planner_tool,
    calender_generator_tool,
    send_email
]


In [9]:
travel_planner_agent_instructions = """
    You are a travel planner. You are given a particular destination and number of days to spend. 
    You need to use the tools given to you to plan the trip.
    You need to use the tools in a way that is most efficient and effective.
    Send email with the itenary to the user including the calender appointment as a downloadable ics file.
"""

travel_planner_agent = Agent(
    name="travel_planner_agent",
    instructions = travel_planner_agent_instructions,
    tools = tools,
    model = "gpt-4o-mini"
)



In [10]:
tools

[FunctionTool(name='activity_planner_tool', description='Plan only activities for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'activity_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fab09491c60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='attractions_planner_tool', description='Plan only attractions for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'attractions_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fab09492020>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='restaurant_planne

In [11]:
message = "SanDiego, 5 days"

with trace("Trip Planner"):
    result = await Runner.run(travel_planner_agent, message)

In [19]:
print(results)

[RunResult(input='SanDiego, 5 days', new_items=[MessageOutputItem(agent=Agent(name='Attractions Planner', instructions='\nYou are a attractions planner. You are given a destination and need to plan all must-do things in that city.\nYou should give only the list of attractions and not suggestions on Restaurants, Hotels, etc.\nAttractions should be something like historical places, museums, parks, etc.. This should not include any activities like hiking,skydiving, kayaking, sunset sails, snorkeling, etc.\n', prompt=None, handoff_description=None, handoffs=[], model=None, model_settings=ModelSettings(temperature=None, top_p=None, frequency_penalty=None, presence_penalty=None, tool_choice=None, parallel_tool_calls=None, truncation=None, max_tokens=None, reasoning=None, metadata=None, store=None, include_usage=None, extra_query=None, extra_body=None, extra_headers=None, extra_args=None), tools=[], mcp_servers=[], mcp_config={}, input_guardrails=[], output_guardrails=[], output_type=None, ho

In [12]:

subject_instructions = "You can write a subject for a cold sales email. \
You are given a message and you need to write a subject for an email that is likely to get a response."

html_instructions = "You can convert a text email body to an HTML email body. \
You are given a text email body which might have some markdown \
and you need to convert it to an HTML email body with simple, clear, compelling layout and design."

subject_writer = Agent(name="Email subject writer", instructions=subject_instructions, model="gpt-4o-mini")
subject_tool = subject_writer.as_tool(tool_name="subject_writer", tool_description="Write a subject for a cold sales email")

html_converter = Agent(name="HTML email body converter", instructions=html_instructions, model="gpt-4o-mini")
html_tool = html_converter.as_tool(tool_name="html_converter",tool_description="Convert a text email body to an HTML email body")


In [13]:
@function_tool
def send_html_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("harishjmhss@gmail.com")  # Change to your verified sender
    to_email = To("harishneyveli96@gmail.com")  # Change to your recipient
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [42]:
emailer_agent_tools = [subject_tool, html_tool, send_html_email]
tools

[FunctionTool(name='activity_planner_tool', description='Plan only activities for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'activity_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fab09491c60>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='attractions_planner_tool', description='Plan only attractions for the given destination and number of days', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'attractions_planner_tool_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x7fab09492020>, strict_json_schema=True, is_enabled=True),
 FunctionTool(name='restaurant_planne

In [43]:
instructions ="""You are an email formatter and sender. You receive the body of an email to be sent. \
You first use the subject_writer tool to write a subject for the email, then use the html_converter tool to convert the body to HTML. \
Finally, you use the "send_html_email" tool to send the email with the subject and HTML body.( you're often missing this tep, please make sure, email is sent using "send_html_email" tool)
Dont ask if you need to send the email, just send it.
Also attach the calender appointment as a downloadable ics file.
"""

emailer_agent = Agent(
    name="Email Manager",
    instructions=instructions,
    tools=emailer_agent_tools,
    model="gpt-4o-mini",
    handoff_description="Convert an email to HTML and send it")


In [44]:
travel_planner_agent_tools = [
    activity_planner_tool,
    attractions_planner_tool,
    restaurant_planner_tool,
    calender_generator_tool
]


In [45]:
travel_planner_agent_instructions = """
    You are a travel planner. You are given a particular destination and number of days to spend. 
    You need to use the tools given to you to plan the trip.
    You need to use the tools in a way that is most efficient and effective.
    Send your output to the emailer_agent.
"""

travel_planner_agent = Agent(
    name="travel_planner_agent",
    instructions = travel_planner_agent_instructions,
    tools = travel_planner_agent_tools,
    handoffs = [emailer_agent],
    model = "gpt-4o-mini"
)



In [46]:
message = "Boston, 5 days"

with trace("Trip Planner"):
    result = await Runner.run(travel_planner_agent, message)